In [ ]:
import os

# Set environment variables
os.environ["AZURE_OPENAI_API_KEY"] = ""
os.environ["AZURE_OPENAI_ENDPOINT"] = ""


In [2]:
!pip install -q  youtube-transcript-api langchain-community langchain-openai \
               faiss-cpu tiktoken python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.0/485.0 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 58.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.5/443.5 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.8/810.8 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.3 MB/s eta 0:00:00


In [3]:
pip install youtube-transcript-api==0.6.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.3/622.3 kB 8.6 MB/s eta 0:00:00:00:010:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

# Step 1a - Indexing (Document Ingestion)

In [11]:
video_id = "Gfr50f6ZBvo" # only the ID, not full URL
try:
    # If you don’t care which language, this returns the “best” one
    transcript_list = YouTubeTranscriptApi().fetch(video_id, languages=["en"])

    # # Flatten it to plain text
    transcript = " ".join([snippet.text for snippet in transcript_list])
    # print(transcript)

except TranscriptsDisabled:
    print("No captions available for this video.")



# Step 1b - Indexing (Text Splitting)

In [12]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

In [13]:
len(chunks)

168

In [14]:
chunks[1]

Document(metadata={}, page_content="out our sponsors in the description and now dear friends here's demis hassabis let's start with a bit of a personal question am i an ai program you wrote to interview people until i get good enough to interview you well i'll be impressed if if you were i'd be impressed by myself if you were i don't think we're quite up to that yet but uh maybe you're from the future lex if you did would you tell me is that is that a good thing to tell a language model that's tasked with interviewing that it is in fact um ai maybe we're in a kind of meta turing test uh probably probably it would be a good idea not to tell you so it doesn't change your behavior right this is a kind of heisenberg uncertainty principle situation if i told you you behave differently yeah maybe that's what's happening with us of course this is a benchmark from the future where they replay 2022 as a year before ais were good enough yet and now we want to see is it going to pass exactly if i

# Step 1c & 1d - Indexing (Embedding Generation and Storing in Vector Store)

In [15]:
embeddings = AzureOpenAIEmbeddings(model="text-embedding-ada-002", api_version='2024-12-01-preview')
vector_store = FAISS.from_documents(chunks, embeddings)

In [16]:
vector_store.index_to_docstore_id

{0: '2d8ae42c-0f0a-40f1-8019-6273311854d1',
 1: '06cbaa13-28e7-43d5-bcc5-bd689af7d89e',
 2: 'f046ed71-aabc-43ed-b239-61fde2ab23a2',
 3: 'e8efc652-387d-4141-bb01-954cae9c82d0',
 4: '9d6cfdbd-7f99-423b-bd63-988f6507c5be',
 5: '1fba4a94-4d6e-4c45-87a9-228373d668d0',
 6: '9b35219b-a491-4d5a-b708-b796b014b56b',
 7: 'e86db18b-f92f-490e-a23e-dad51b9cada9',
 8: 'f933187e-77d1-48c4-a57d-8e9522315251',
 9: '9aef74b2-2dc0-4dda-876c-ed68b07da88d',
 10: '1e071dea-8084-4792-975e-7ddd3ffe3e85',
 11: 'c9b6d2aa-3bfd-44db-b133-87ab21c9e975',
 12: '369c348e-9301-4710-bd4b-aeb2982ccdc1',
 13: 'b7e8952c-57f6-4578-869d-e6752a1ca529',
 14: '79c260d8-1e2a-44ef-8f6b-00def0d2de5b',
 15: '174ee3a7-e585-43a5-9196-c76e0503c8be',
 16: '359bae91-f881-4df9-a05e-9b6f20445ddb',
 17: 'c87ac36d-d3a5-4871-9270-7bcc0af9dbba',
 18: 'bb713fe7-323c-4b1d-8901-54dae46e1874',
 19: '44fb6d50-3b2c-4771-aead-cc87049ae33e',
 20: 'a43851cb-5b64-4ed5-b562-b77bc4642c04',
 21: '5b3916c1-9948-4ec3-8e7d-d3c8f3d94d12',
 22: 'cf0f91e5-43d8-

In [18]:
vector_store.get_by_ids(['4db0a17c-af68-4414-9fae-ac9ca741c97b'])

[Document(id='4db0a17c-af68-4414-9fae-ac9ca741c97b', metadata={}, page_content='demas establish to support this podcast please check out our sponsors in the description and now let me leave you with some words from edskar dykstra computer science is no more about computers than astronomy is about telescopes thank you for listening and hope to see you next time')]

# Step 2 - Retrieval

In [19]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [20]:
retriever

VectorStoreRetriever(tags=['FAISS', 'AzureOpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7943a1e13490>, search_kwargs={'k': 4})

In [21]:
retriever.invoke('What is deepmind')

[Document(id='2d8ae42c-0f0a-40f1-8019-6273311854d1', metadata={}, page_content="the following is a conversation with demus hasabis ceo and co-founder of deepmind a company that has published and builds some of the most incredible artificial intelligence systems in the history of computing including alfred zero that learned all by itself to play the game of gold better than any human in the world and alpha fold two that solved protein folding both tasks considered nearly impossible for a very long time demus is widely considered to be one of the most brilliant and impactful humans in the history of artificial intelligence and science and engineering in general this was truly an honor and a pleasure for me to finally sit down with him for this conversation and i'm sure we will talk many times again in the future this is the lex friedman podcast to support it please check out our sponsors in the description and now dear friends here's demis hassabis let's start with a bit of a personal qu

# Step 3 - Augmentation

In [40]:
llm = AzureChatOpenAI(
    deployment_name="gpt-5-nano",  
    openai_api_version="2024-12-01-preview")

In [23]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [32]:
question          = "is the topic of nuclear fusion discussed in this video? if yes then what was discussed"
retrieved_docs    = retriever.invoke(question)

In [33]:
retrieved_docs

[Document(id='48829b24-8eca-4a39-b479-48d2c77c8f10', metadata={}, page_content="in this case in fusion we we collaborated with epfl in switzerland the swiss technical institute who are amazing they have a test reactor that they were willing to let us use which you know i double checked with the team we were going to use carefully and safely i was impressed they managed to persuade them to let us use it and um and it's a it's an amazing test reactor they have there and they try all sorts of pretty crazy experiments on it and um the the the what we tend to look at is if we go into a new domain like fusion what are all the bottleneck problems uh like thinking from first principles you know what are all the bottleneck problems that are still stopping fusion working today and then we look at we you know we get a fusion expert to tell us and then we look at those bottlenecks and we look at the ones which ones are amenable to our ai methods today yes right and and and then and would be intere

In [36]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"in this case in fusion we we collaborated with epfl in switzerland the swiss technical institute who are amazing they have a test reactor that they were willing to let us use which you know i double checked with the team we were going to use carefully and safely i was impressed they managed to persuade them to let us use it and um and it's a it's an amazing test reactor they have there and they try all sorts of pretty crazy experiments on it and um the the the what we tend to look at is if we go into a new domain like fusion what are all the bottleneck problems uh like thinking from first principles you know what are all the bottleneck problems that are still stopping fusion working today and then we look at we you know we get a fusion expert to tell us and then we look at those bottlenecks and we look at the ones which ones are amenable to our ai methods today yes right and and and then and would be interesting from a research perspective from our point of view from an ai point of\n\

In [37]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [38]:
final_prompt

StringPromptValue(text="\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      in this case in fusion we we collaborated with epfl in switzerland the swiss technical institute who are amazing they have a test reactor that they were willing to let us use which you know i double checked with the team we were going to use carefully and safely i was impressed they managed to persuade them to let us use it and um and it's a it's an amazing test reactor they have there and they try all sorts of pretty crazy experiments on it and um the the the what we tend to look at is if we go into a new domain like fusion what are all the bottleneck problems uh like thinking from first principles you know what are all the bottleneck problems that are still stopping fusion working today and then we look at we you know we get a fusion expert to tell us and then we look at those bottlenecks and we 

# Step 4 - Generation

In [41]:
answer = llm.invoke(final_prompt)
print(answer.content)

Yes. The video discusses nuclear fusion and includes:

- Collaboration with EPFL (the Swiss Federal Institute of Technology) in Switzerland and access to their test reactor.
- The approach of identifying bottleneck problems in fusion from first principles and determining which could be tackled with AI.
- A Nature paper on using deep reinforcement learning to magnetic-control tokamak plasmas, showing the plasma can be carved into specific shapes and held there for a record amount of time; shapes mentioned include droplets and others.
- Plans to work with fusion startups to tackle the next problems in fusion.
- Related AI/physics work, such as pushing the frontiers of density functionals by solving the fractional electron problem.
- The view that energy and climate are major domains for AI’s impact, with fusion as a key area.


# Building a Chain

In [42]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [43]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [44]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [45]:
parallel_chain.invoke('who is Demis')

{'context': "the following is a conversation with demus hasabis ceo and co-founder of deepmind a company that has published and builds some of the most incredible artificial intelligence systems in the history of computing including alfred zero that learned all by itself to play the game of gold better than any human in the world and alpha fold two that solved protein folding both tasks considered nearly impossible for a very long time demus is widely considered to be one of the most brilliant and impactful humans in the history of artificial intelligence and science and engineering in general this was truly an honor and a pleasure for me to finally sit down with him for this conversation and i'm sure we will talk many times again in the future this is the lex friedman podcast to support it please check out our sponsors in the description and now dear friends here's demis hassabis let's start with a bit of a personal question am i an ai program you wrote to interview people until i get

In [46]:
parser = StrOutputParser()

In [47]:
main_chain = parallel_chain | prompt | llm | parser

In [48]:
main_chain.invoke('Can you summarize the video')

'- The video is a Lex Fridman Podcast episode featuring a conversation with Demis Hassabis, CEO and co-founder of DeepMind.\n- They discuss DeepMind’s groundbreaking AI work (mentioning AlphaGo Zero and AlphaFold 2) and Hassabis’s views on intelligence and problem-solving.\n- A core theme is the idea that true understanding may come from simple, clear explanations; they reference this as a sign of intelligence, linking it to Richard Feynman.\n- They talk about gaps in current physics (like the standard model) and how better explanations could help illuminate big mysteries such as consciousness, life, and gravity.\n- The conversation touches on the relationship between humans and computation, including a potential symbiosis with devices and AI.\n- There are reflections on what makes something truly intelligent—namely, the ability to explain it simply.\n- The episode includes sponsor messages and ends with a nod to Edsger Dijkstra’s quote about computer science not being about computers.